In [ ]:
import re

def remove_markup_tags(text):
    clean_text = re.sub('<(/?)mark>', '', text)
    return clean_text

def find_position_in_original(markup_text, position):
    clean_text = remove_markup_tags(markup_text)
    result = 0
    index = 0
    start = "<mark>"
    end = "</mark>"
    while index != position:
        if markup_text[result:result+len(start)] == start:
            result += len(start)
        elif markup_text[result:result+len(end)] == end:
            result += len(end)
        else:
            result += 1
            index += 1
        if result == len(markup_text):
            break
    return result

def first_diff_index(str1, str2):
    for index, (char1, char2) in enumerate(zip(str1, str2)):
        if char1 != char2:
            return index
    return min(len(str1), len(str2))

def last_occurrence_before_index(x, substring, index):
    return x.rfind(substring, 0, index)

def modified_marked_up_string(markup_text, edited_clean_text):
    clean_text = remove_markup_tags(markup_text)
    edit_index = first_diff_index(clean_text, edited_clean_text)
    markup_text_pos = find_position_in_original(markup_text, edit_index)

    start_pos = markup_text.rfind("<mark>", 0, markup_text_pos)
    end_pos = markup_text.rfind("</mark>", 0, markup_text_pos)
    if start_pos < end_pos:
        modified_text = markup_text[:markup_text_pos] + edited_clean_text[edit_index:]
    else:
        modified_text = markup_text[:markup_text_pos] + "</mark>" + edited_clean_text[edit_index:]

    return modified_text

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import openai
import re

# Replace 'your_api_key_here' with your OpenAI API key
openai.api_key = "your_api_key_here"

markedup_context = ""

def on_submit_button_click(button):
    global markedup_context
    # Get the current context
    context = text_editor.value
    markedup_context = modified_marked_up_string(markedup_context, context)

    # Call OpenAI API with the context
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "assistant", "content": context}],
        max_tokens=max_tokens_slider.value,
        n=1,
        temperature=temperature_slider.value
    )

    # Get the response from OpenAI and append it to the context
    generated_text = response['choices'][0]['message']['content']
    text_editor.value = context + " " + generated_text
    markedup_context += " <mark>" + generated_text + "</mark>"
    render = (markedup_context.replace("<mark>", '<span style="background-color:#ccffcc;">')
                              .replace("</mark>", "</span>")
                              .replace("\n","<br>"))
    
    html_output.value = render

      
def on_submit_button_clickv2(button):
    global markedup_context
    # Get the current context
    context = text_editor.value
    markedup_context = modified_marked_up_string(markedup_context, context)

    # Call OpenAI API with the context
    markedup_context += "<mark>"
    for response in openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "assistant", "content": context}],
        max_tokens=max_tokens_slider.value,
        n=1,
        temperature=temperature_slider.value,
        stream=True):
        # Get the response from OpenAI and append it to the context
        #print(response['choices'][0])
        
        try:
            generated_text = response['choices'][0]['delta']['content']
        except:
            if response['choices'][0]['finish_reason'] == 'length':
                generated_text = ""
            else:
                generated_text = " "
            
        text_editor.value += generated_text
        markedup_context += generated_text
        render = (markedup_context.replace("<mark>", '<span style="background-color:#ccffcc;">')
                  .replace("</mark>", "</span>")
                  .replace("\n","<br>")
                 ) + "</span>"
        html_output.value = render
        #markdown_output.data = render
    markedup_context += "</mark>"

    
# Set up text editor
text_editor = widgets.Textarea(
    value='',
    placeholder='Type your text here...',
    description='',
    disabled=False,
    rows=25,  # Adjust the number of rows to display (height)
    layout=widgets.Layout(width='80%')  # Adjust the width as a percentage of the container width
)

html_output = widgets.HTML(
    value='',
    placeholder='Type your text here...',
    description='',
    disabled=False
)

markdown_output = Markdown('')

# Set up submit button
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',
    tooltip='Submit the context to OpenAI',
    icon='check'
)

submit_button.on_click(on_submit_button_clickv2)

# Set up temperature slider
temperature_slider = widgets.FloatSlider(
    value=0.8,
    min=0.1,
    max=1.0,
    step=0.1,
    description='Temperature:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Set up max_tokens slider
max_tokens_slider = widgets.IntSlider(
    value=50,
    min=1,
    max=512,
    step=1,
    description='Max tokens:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

In [ ]:
display(html_output)
display(text_editor)
display(submit_button)
display(temperature_slider)
display(max_tokens_slider)